In [1]:
import sys
sys.path.append('.')
sys.path.append('..')
from problem_loader import ProblemLoader
from helpers import obfuscate

data_urls = {
    'tsp': 'https://d18ky98rnyall9.cloudfront.net/_f702b2a7b43c0d64707f7ab1b4394754_tsp.txt?Expires=1629504000&Signature=LEM~cWnuYcjXjwVrDUoSeppI6n~XSXGnBAUIULaP6SKwkvMafbNR2t~cCC0d0FwcBru51~1kuOu-weoGbcZp2CQtUc-0pDGeCbpkIbXx7pkusqtYbHcvIDAHovIbMUMCJl5K5xvtHClg8wm~XjUxDI2QDHaU9YzMlqqP42d3XyY_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'
}

# Problem 1

In this assignment you will implement one or more algorithms for the traveling salesman problem, such as the dynamic programming algorithm covered in the video lectures.  

The first line indicates the number of cities.  Each city is a point in the plane, and each subsequent line indicates the x- and y-coordinates of a single city.  

The distance between two cities is defined as the Euclidean distance --- that is, two cities at locations $(x,y)$ and $(z,w)$ have distance $\sqrt{(x-z)^2 + (y-w)^2}$​ between them.  

In the box below, type in the minimum cost of a traveling salesman tour for this instance, rounded down to the nearest integer.

### OPTIONAL: 
If you want bigger data sets to play with, check out the TSP instances from around the world here.  The smallest data set (Western Sahara) has 29 cities, and most of the data sets are much bigger than that.  What's the largest of these data sets that you're able to solve --- using dynamic programming or, if you like, a completely different method?

### HINT: 
You might experiment with ways to reduce the data set size.  For example, trying plotting the points.  Can you infer any structure of the optimal solution?  Can you use that structure to speed up your algorithm?

In [2]:
def process_cities(data):
  v = []
  for edge in data.split(b'\n'):
    sa = edge.decode('utf-8').split(' ')
    if len(sa) > 1:
        v.append((float(sa[0]), float(sa[1])))
  return v

cities = ProblemLoader(
    data_urls['tsp'], 
    fname="tsp.p", 
    preprocessor=process_cities,
).fetch()
print(cities)

[(20833.3333, 17100.0), (20900.0, 17066.6667), (21300.0, 13016.6667), (21600.0, 14150.0), (21600.0, 14966.6667), (21600.0, 16500.0), (22183.3333, 13133.3333), (22583.3333, 14300.0), (22683.3333, 12716.6667), (23616.6667, 15866.6667), (23700.0, 15933.3333), (23883.3333, 14533.3333), (24166.6667, 13250.0), (25149.1667, 12365.8333), (26133.3333, 14500.0), (26150.0, 10550.0), (26283.3333, 12766.6667), (26433.3333, 13433.3333), (26550.0, 13850.0), (26733.3333, 11683.3333), (27026.1111, 13051.9444), (27096.1111, 13415.8333), (27153.6111, 13203.3333), (27166.6667, 9833.3333), (27233.3333, 10450.0)]


In [3]:
__test__ = True
if __test__:
  """
  3
0 0
0 3
3 3
// 10.24
  # cities = [(0,0), (0,3), (3,3)]  
  """
  """
  8
0 2.05
3.414213562373095 3.4642135623730947
0.5857864376269049 0.6357864376269047
0.5857864376269049 3.4642135623730947
2 0
4.05 2.05
2 4.10
3.414213562373095 0.6357864376269047
// 12.36
  # cities = [(0,2.05), (3.414213562373095, 3.4642135623730947), (0.5857864376269049, 0.6357864376269047), (0.5857864376269049, 3.4642135623730947), (2, 0), (4.05, 2.05), (2, 4.10), (3.414213562373095, 0.6357864376269047)]
  """
  """
  4
0 0
4 3
4 0
0 3
// 14.00
  # cities = [(0,0), (4,3), (4,0), (0,3)]
  """
  """
12.00
1 1
1.125 1
1.25 1
1.5 1
1.75 1
2 1
1 2
1.125 2
1.25 2
1.5 2
1.75 2
2 2
// 4.00
#  cities = [(1, 1), (1.125, 1), (1.25, 1), (1.5, 1), (1.75, 1), (2, 1), (1, 2), (1.125, 2), (1.25, 2), (1.5, 2), (1.75, 2), (2, 2)] 
  """

### lemma: 2-Change
Given a tour $T$, remove two edges $(v, w), (u, x)$ of $T$  that do not share an endpoint.  
Add either the edges $(v, x)$ and $(u, w)$ or the edges $(u, v)$ and $(w, x)$, whichever pair leads to a new tour $T_0$. 

## 2-OPT

#### Input: 
complete graph $G = (V,E)$ and cost $c_e$ for each edge $e \in E$.  

### Output: 
a traveling salesman tour.  

$T :=$ initial tour `// perhaps greedily constructed`  
while improving 2-change $(v, w),(u, x) \in T$ exists do  
&nbsp; $T := 2Change(T,(v, w),(u, x))$  
return $T$

In [4]:
def get_distance(left, right):
  """ return the euclidean distance between tuples left and right, which are coordinates"""
  return ((left[0] - right[0]) ** 2 + (left[1] - right[1]) ** 2) ** 0.5

# precompute all distances
distances = {}
for i, city in enumerate(cities):
  for j, city2 in enumerate(cities):
    if j <= i:
      continue
    distances[(i, j)] = get_distance(city, city2)
    distances[(j, i)] = get_distance(city, city2)

print(distances)

{(0, 1): 74.53561415712697, (1, 0): 74.53561415712697, (0, 2): 4109.913459889123, (2, 0): 4109.913459889123, (0, 3): 3047.9957068357057, (3, 0): 3047.9957068357057, (0, 4): 2266.911731360042, (4, 0): 2266.911731360042, (0, 5): 973.5388173508504, (5, 0): 973.5388173508504, (0, 6): 4190.100799370928, (6, 0): 4190.100799370928, (0, 7): 3301.893396219811, (7, 0): 3301.893396219811, (0, 8): 4757.742197606854, (8, 0): 4757.742197606854, (0, 9): 3044.3481805543315, (9, 0): 3044.3481805543315, (0, 10): 3094.978054490498, (10, 0): 3094.978054490498, (0, 11): 3986.2611491081325, (11, 0): 3986.2611491081325, (0, 12): 5092.505430095836, (12, 0): 5092.505430095836, (0, 13): 6406.149567403533, (13, 0): 6406.149567403533, (0, 14): 5903.388857258176, (14, 0): 5903.388857258176, (0, 15): 8436.198480292465, (15, 0): 8436.198480292465, (0, 16): 6962.778000833352, (16, 0): 6962.778000833352, (0, 17): 6693.612230245258, (17, 0): 6693.612230245258, (0, 18): 6575.92412964816, (18, 0): 6575.92412964816, (0, 1

In [5]:
class Leg:
  """ a leg of the trip is defined from the perspective of the origin. The cost of the leg 
  of the trip denotes the distance to the next stop in the trip """
  def __init__(self, coords, next=None) -> None:
    self.coords = coords
    self.next = next

  def cost(self, start=None):
    """ return the cost of a tour starting at initial"""
    global distances
    if start is None:
      start = self
    elif start.coords == self.coords: # halt before we start running laps
      return 0

    if self.coords == self.next.coords:
      print(self)

    return distances[(cities.index(self.coords), cities.index(self.next.coords))] \
      + self.next.cost(start=start)

  def trace(self, start=None):
    """return a list of coordinates, stopping at the first lap (before repeating)"""
    if start is None:
      start = self
    elif start.coords == self.coords:
      return []

    return [self.coords] + self.next.trace(start=start)

  def indexed_composition(self, start=None):
    """return a list of each leg in the trip"""
    if start is None:
      start = self
    elif start.coords == self.coords:
      return []

    return [self] + self.next.indexed_composition(start=start)

  def copy(self):
    """return a copy of the tour"""
    i = 0
    trace = self.trace()
    tour = Leg(trace[i])
    candidate_tour = tour
    visited = [trace[i]]
    while len(trace) > len(visited):
      i += 1
      next = trace[i]
      candidate_tour.next = Leg(next)
      candidate_tour = candidate_tour.next
      visited.append(next)
    candidate_tour.next = tour

    return tour


  def __str__(self):
    return f"({self.coords} -> {self.next.coords})"


In [6]:
from math import inf
from itertools import combinations, permutations
from memoization import cached as memoize

@memoize(thread_safe=False)
def swap_n(tour, indices):
  """ swap coordinates at n indices of a tour """
  best_tour = None
  best_cost = inf

  n_aries = list(permutations(indices, len(indices)))
  # print('n_aries', n_aries, indices)

  for new_pos in n_aries:
    if new_pos == indices:
      continue

    new_tour = tour.copy()
    # gather legs from new_pos
    t = new_tour.trace()
    replacements = [t[i] for i in new_pos]
    ic = new_tour.indexed_composition() # get a list of each link in the linked-list

    for i, from_place in enumerate(indices):
      ic[from_place].coords = replacements[i]

    c = new_tour.cost()
    if c < best_cost:
      best_cost = c
      best_tour = new_tour.copy()

  return best_tour


In [7]:
def nearest_neightbor(graph, node, exclude=[]):
  """ return the nearest neighbor of node in graph, except for nodes in exclusion list """
  i = cities.index(node)
  return min(filter(lambda x: x not in exclude, graph), key=lambda x: distances[(i, cities.index(x))])

In [8]:
# get initial_tour, including every city in the graph
tour = Leg(cities[0])
candidate_tour = tour
visited = [cities[0]]
while len(cities) > len(visited):
  next = nearest_neightbor(cities, visited[-1], exclude=visited)
  candidate_tour.next = Leg(next)
  candidate_tour = candidate_tour.next
  visited.append(next)
candidate_tour.next = tour

print(tour.trace(), tour.cost(), flush=True)

goal = tour.cost() 
#pairs = list(combinations(range(len(cities)), 2))
#triples = list(combinations(range(len(cities)), 3))
#quadruples = list(combinations(range(len(cities)), 4))
LIMIT = 4 # 7 is required to solve from the greedy tour we start with

@memoize(thread_safe=False)
def opt(initial, tour):
  """ use n-opt (2&3) to get progressively better tours """
  global pairs, triples, LIMIT

  goal = initial
  next_tour = None

  for size in range(2, LIMIT):
    duples = list(combinations(range(len(cities)), size))

    for args in duples:
      candidate_tour = swap_n(tour, list(args))
      value = candidate_tour.cost()
      if value < goal:
        goal = value
        next_tour = candidate_tour.copy()

    if next_tour != None:
      print(initial, goal, flush=True)
      return opt(goal, next_tour)
  
  return tour

def main():
  global goal, tour
  tour = opt(goal, tour)
  goal = tour.cost()
  print(tour.trace(), goal)



[(20833.3333, 17100.0), (20900.0, 17066.6667), (21600.0, 16500.0), (21600.0, 14966.6667), (21600.0, 14150.0), (22583.3333, 14300.0), (22183.3333, 13133.3333), (22683.3333, 12716.6667), (21300.0, 13016.6667), (24166.6667, 13250.0), (23883.3333, 14533.3333), (23616.6667, 15866.6667), (23700.0, 15933.3333), (26133.3333, 14500.0), (26550.0, 13850.0), (26433.3333, 13433.3333), (27096.1111, 13415.8333), (27153.6111, 13203.3333), (27026.1111, 13051.9444), (26283.3333, 12766.6667), (26733.3333, 11683.3333), (26150.0, 10550.0), (27233.3333, 10450.0), (27166.6667, 9833.3333), (25149.1667, 12365.8333)] 32981.96301468097


In [9]:
main()

32981.96301468097 31922.266378509827
31922.266378509827 31383.197488540885
31383.197488540885 30969.89035836883
30969.89035836883 30745.981312029362
30745.981312029362 30562.91972745309
30562.91972745309 30291.731547257266
30291.731547257266 29268.055435213675
29268.055435213675 29119.549961875276
29119.549961875276 28976.389447516092
28976.389447516092 28956.78509278778
[(24166.6667, 13250.0), (21600.0, 16500.0), (20900.0, 17066.6667), (20833.3333, 17100.0), (21600.0, 14966.6667), (21600.0, 14150.0), (21300.0, 13016.6667), (22183.3333, 13133.3333), (22683.3333, 12716.6667), (22583.3333, 14300.0), (23883.3333, 14533.3333), (23616.6667, 15866.6667), (23700.0, 15933.3333), (26133.3333, 14500.0), (26550.0, 13850.0), (26433.3333, 13433.3333), (27096.1111, 13415.8333), (27153.6111, 13203.3333), (27026.1111, 13051.9444), (26283.3333, 12766.6667), (26733.3333, 11683.3333), (27233.3333, 10450.0), (27166.6667, 9833.3333), (26150.0, 10550.0), (25149.1667, 12365.8333)] 28956.78509278778
